In [3]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plot

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# CIFAR_10 is a set of 60k images 32x32 pixels on 3 channels
# Sets the number of color channels in the input images.
# 3 is for RGB
IMG_CHANNELS = 3

# sets width to 32px
IMG_ROWS = 32

# set the height to 32px
IMG_COLS = 32

#constant

# is the number of samples that will be propagated
# through the neural network at once.
BATCH_SIZE = 128
# number of rounds the data will be passed 
# through the neural network
NB_EPOCH = 20
# Classes or categories tthe input data can be
# classified into.
NB_CLASSES = 10
# Controls the amount of information printed
# during the training process. -- cool but not necessary -- 
VERBOSE = 1
# This line sets the fraction of the training data
# that will be used for validation during training.
#  In this case, 20% of the training
# data will be used for validation.
VALIDATION_SPLIT = 0.2
# the optimizer algorithm is RMSprop
OPTIM = RMSprop()



#load dataset

# mapping the elements of the image data to x values with labels
# and y values with labels
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert to categorical
# Convert the class labels from integers to one-hot
# encoded vectors.
# first index points the item the second index points to some other
# posiont 
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

## Converts the results to float32 integers (floating-point numbers)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# does some weird division by 255
# Normalize them by dividing by 255 (the maximum pixel value).
#  This is a common preprocessing step for image data
# as it helps improve the numerical stability and
# convergence of the neural network. -- SO MANY QUESTIONS !!! ---
X_train /= 255
X_test /= 255

# network
# initializes the model
model = Sequential()

# Layer One
# This is the first convolutional layer with 32 filters of size 3x3
# The padding is set to 'same' to ensure that the output has the same spatial dimensions as the input
# The input shape is specified as (IMG_ROWS, IMG_COLS, IMG_CHANNELS)
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))

# Activation function for the first convolutional layer (ReLU)
model.add(Activation('relu'))

# Second convolutional layer with 32 filters of size 3x3
# The padding is set to 'same' to ensure that the output has the same spatial dimensions as the input
model.add(Conv2D(32, kernel_size=(3, 3), padding='same'))

# Activation function for the second convolutional layer (ReLU)
model.add(Activation('relu'))

# Max pooling layer with a 2x2 pool size
# This downsamples the feature maps and introduces translation invariance -- explain...
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout layer with a rate of 0.25 (25% of the inputs will be randomly set to 0)
# Dropout is a regularization technique to prevent overfitting -- explain...
model.add(Dropout(0.25))

# End of Layer One

# Layer Two
# Third convolutional layer with 64 filters of size 3x3
# The padding is set to 'same' to ensure that the output has the same spatial dimensions as the input
model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))

# Activation function for the third convolutional layer (ReLU)
model.add(Activation('relu'))

# Fourth convolutional layer with 64 filters of size 3x3
# No padding specified, so the output spatial dimensions may be reduced
model.add(Conv2D(64, kernel_size=(3, 3)))

# Activation function for the fourth convolutional layer (ReLU)
model.add(Activation('relu'))

# cleanup

# Cleanup/Pooling Layer
# Max pooling layer with a 2x2 pool size
# This downsamples the feature maps and introduces translation invariance
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout layer with a rate of 0.25 (25% of the inputs will be randomly set to 0)
# Dropout is a regularization technique to prevent overfitting -- dont work too hard 
model.add(Dropout(0.25))

# Flatten Layer
# Flattens the multi-dimensional output from the convolutional and pooling layers
# into a 1D vector, which is required for the fully connected layers -- thats cray
model.add(Flatten())

# Fully Connected Layer 1
# Fully connected layer with 512 units
# This layer combines and processes the features learned by the convolutional and pooling layers
model.add(Dense(512))

# Activation function for the fully connected layer (ReLU)
model.add(Activation('relu'))

# Dropout layer with a rate of 0.5 (50% of the inputs will be randomly set to 0)
# Dropout is a regularization technique to prevent overfitting
model.add(Dropout(0.5))

# Output Layer
# Fully connected layer with the same number of units as the number of classes (NB_CLASSES)
# This is the final layer that produces the output predictions
model.add(Dense(NB_CLASSES))

# Softmax activation function for the output layer
# Softmax is commonly used in multi-class classification problems
# It outputs a probability distribution over the classes, where the sum of probabilities is 1
model.add(Activation('softmax'))

# results
model.summary()

ImportError: cannot import name 'np_utils' from 'keras.utils' (/Users/petertumulty/.pyenv/versions/3.9.12/lib/python3.9/site-packages/keras/utils/__init__.py)

In [ ]:
# train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM,
metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
score = model.evaluate(X_test, Y_test,
batch_size=BATCH_SIZE, verbose=VERBOSE)

print("Test score:", score[0])
print('Test accuracy:', score[1])

## How can training machines on images be harmful? What are the negative impacts of the technology on our society*

Training machines on images can be harmful because they inherently introduce bias by design. Beyond training the DCNN on biased data, the pooling layer, specifically the max pooling layer, starts a process that is inherently biased. I came to this conclusion after reviewing the code from Chapter 3 of "Deep Learning with Keras". To my understanding, max pooling is introduced into the script, designed to prioritize some features of the element over others. This process corrupts the input data in some way. For example, it can make the data insensitive to small shifts, which can result in biased data. "In Max Pooling, small changes in the non-maximal values will be ignored. This insensitivity to small changes can be problematic and can bias the results (Arham, 2023)." When we introduce favoritism into the training data at that core level of the DCNN, the output will always exhibit some degree of bias, regardless of the input. This change can emphasis the negative aspects of the input data unintentionally. 


## For example, could such an algorithm eventually be used to distinguish people’s faces? If so, what are the ethical and privacy implications?

When passing image data to a neural network, it's important to consider the potential negative effects, particularly from an ethical standpoint. For instance, feeding images of people into a Deep Convolutional Neural Network (DCNN) prompts the algorithm to analyze and interpret these images. If the developers of the DCNN persistently refine the system to recreate images with high accuracy and likeness based on the input data, this can lead to significant ethical concerns. One problem based on this fact, which is also is becoming more prevalent in our society: deepfakes, which are "kind of synthetic media where a person in an image or video is swapped with another person’s likeness (Dhiman, 2023)." Even though the code we ran seemed pretty interesting at the time, it could easily be refactored to generate deepfake photos based on the training data provided, raising further a very important ethical concerns.

There isn't much argument against how deepfakes are unethical and invade personal privacy. A recent example of the impact how damaging deepfakes can be is what happened to Taylor Swift. Taylor Swift was a victim of deepfake with her face imposed over pornographic images and shared on social media (Associated Press, 2024). As of right now the source of these images is unknown, but whoever is behind it they are clearly wrong morally and legally.  Since the incident according to the BBC, "US politicians have called for new laws to criminalise the creation of deepfake(Rahman-Jones, 2024)" based on this incident. This case highlights the urgent need for legislation that specifically targets deepfake technology to protect individuals' rights and privacy.

**References:**

Arham, M. (2023, September 28). Diving into the pool: Unraveling the magic of CNN pooling layers. KDnuggets. Retrieved from [https://www.kdnuggets.com/diving-into-the-pool-unraveling-the-magic-of-cnn-pooling-layers](https://www.kdnuggets.com/diving-into-the-pool-unraveling-the-magic-of-cnn-pooling-layers)

Dhiman, B. (2023, December 7). Exploding AI-generated deepfakes and misinformation: A threat to global concern in the 21st century. ResearchGate. [https://www.researchgate.net/publication/376329868_Exploding_AI-Generated_Deepfakes_and_Misinformation_A_Threat_to_Global_Concern_in_the_21st_Century](https://www.researchgate.net/publication/376329868_Exploding_AI-Generated_Deepfakes_and_Misinformation_A_Threat_to_Global_Concern_in_the_21st_Century)

Associated Press. (2024, January 26). Fans fight back as fake explicit images of Taylor Swift spread on social media. MSN. [https://www.msn.com/en-us/news/technology/fans-fight-back-as-fake-explicit-images-of-taylor-swift-spread-on-social-media/ar-BB1hjPDV](https://www.msn.com/en-us/news/technology/fans-fight-back-as-fake-explicit-images-of-taylor-swift-spread-on-social-media/ar-BB1hjPDV)

Rahman-Jones, I. (2024, January 27). Taylor Swift deepfakes spark calls in Congress for new legislation. BBC News. [https://www.bbc.com/news/technology-68110476?embed=true](https://www.bbc.com/news/technology-68110476?embed=true)